# సెషన్ 2 – రాగాలతో RAG మూల్యాంకనం

రాగాస్ మెట్రిక్స్ ఉపయోగించి కనిష్ట RAG పైప్‌లైన్‌ను మూల్యాంకనం చేయండి: answer_relevancy, faithfulness, context_precision.


# సన్నివేశం
ఈ సన్నివేశం ఒక కనిష్ట Retrieval Augmented Generation (RAG) పైప్‌లైన్‌ను స్థానికంగా మూల్యాంకనం చేస్తుంది. మేము:
- ఒక చిన్న సింథటిక్ డాక్యుమెంట్ కార్పస్‌ను నిర్వచిస్తాము.
- డాక్స్‌ను ఎంబెడ్ చేసి, ఒక సాధారణ సారూప్యత రిట్రీవర్‌ను అమలు చేస్తాము.
- స్థానిక మోడల్ (Foundry Local / OpenAI-అనుకూల) ఉపయోగించి ఆధారిత సమాధానాలను ఉత్పత్తి చేస్తాము.
- ragas మెట్రిక్స్ (`answer_relevancy`, `faithfulness`, `context_precision`) ను లెక్కిస్తాము.
- వేగవంతమైన పునరావృతానికి (env `RAG_FAST=1`) కేవలం సమాధాన సంబంధితతను మాత్రమే లెక్కించే FAST మోడ్‌ను మద్దతు ఇస్తుంది.

మీ స్థానిక మోడల్ + ఎంబెడ్డింగ్స్ స్టాక్ పెద్ద కార్పస్‌లకు విస్తరించే ముందు వాస్తవానికి ఆధారిత సమాధానాలను ఉత్పత్తి చేస్తున్నదని నిర్ధారించడానికి ఈ నోట్‌బుక్‌ను ఉపయోగించండి.


### వివరణ: డిపెండెన్సీ ఇన్‌స్టాలేషన్
అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేస్తుంది:
- స్థానిక మోడల్ నిర్వహణ కోసం `foundry-local-sdk`.
- `openai` క్లయింట్ ఇంటర్‌ఫేస్.
- సాంద్ర ఎంబెడ్డింగ్స్ కోసం `sentence-transformers`.
- మూల్యాంకనం & మెట్రిక్ లెక్కింపు కోసం `ragas` + `datasets`.
- ragas LLM ఇంటర్‌ఫేస్ కోసం `langchain-openai` అడాప్టర్.

పునఃప్రారంభించడానికి సురక్షితం; వాతావరణం ఇప్పటికే సిద్ధంగా ఉంటే దాటవేయండి.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### వివరణ: కోర్ ఇంపోర్ట్స్ & మెట్రిక్స్
కోర్ లైబ్రరీలు మరియు రాగాస్ మెట్రిక్స్‌ను లోడ్ చేస్తుంది. ముఖ్య భాగాలు:
- ఎంబెడ్డింగ్స్ కోసం SentenceTransformer.
- `evaluate` + ఎంపిక చేసిన రాగాస్ మెట్రిక్స్.
- మూల్యాంకన కార్పస్ నిర్మాణం కోసం `Dataset`.
ఈ ఇంపోర్ట్స్ రిమోట్ కాల్స్‌ను ప్రేరేపించవు (ఎంబెడ్డింగ్స్ కోసం మోడల్ క్యాష్ లోడ్ తప్ప).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### వివరణ: టాయ్ కార్పస్ & QA గ్రౌండ్ ట్రూత్
ఒక చిన్న ఇన్-మెమరీ కార్పస్ (`DOCS`), యూజర్ ప్రశ్నల సెట్, మరియు ఆశించిన గ్రౌండ్ ట్రూత్ సమాధానాలను నిర్వచిస్తుంది. ఇవి బాహ్య డేటా ఫెచ్‌లను లేకుండా వేగవంతమైన, నిర్ణయాత్మక మెట్రిక్ లెక్కింపును అనుమతిస్తాయి. వాస్తవ పరిస్థితుల్లో మీరు ఉత్పత్తి ప్రశ్నలు + శ్రద్ధగా ఎంపిక చేసిన సమాధానాలను నమూనా తీసుకుంటారు.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### వివరణ: సర్వీస్ ఇనిట్, ఎంబెడ్డింగ్స్ & సేఫ్టీ ప్యాచ్
Foundry Local మేనేజర్‌ను ప్రారంభిస్తుంది, `promptTemplate` కోసం స్కీమా-డ్రిఫ్ట్ సేఫ్టీ ప్యాచ్‌ను వర్తింపజేస్తుంది, మోడల్ IDని పరిష్కరిస్తుంది, OpenAI-అనుకూల క్లయింట్‌ను సృష్టిస్తుంది, మరియు డాక్యుమెంట్ కార్పస్ కోసం డెన్స్ ఎంబెడ్డింగ్స్‌ను ముందుగా గణిస్తుంది. ఇది రిట్రీవల్ + జనరేషన్ కోసం పునర్వినియోగపరచదగిన స్థితిని సెట్ చేస్తుంది.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### వివరణ: రిట్రీవర్ ఫంక్షన్
సాధారణ వెక్టర్ సారూప్యత రిట్రీవర్‌ను సాధారణీకరించిన ఎంబెడ్డింగ్స్ పై డాట్ ప్రొడక్ట్ ఉపయోగించి నిర్వచిస్తుంది. టాప్-k డాక్స్ (k=2 డిఫాల్ట్) ను తిరిగి ఇస్తుంది. ఉత్పత్తిలో స్కేల్ & లేటెన్సీ కోసం ANN సూచిక (FAISS, Chroma, Milvus) తో మార్చండి.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### వివరణ: జనరేషన్ ఫంక్షన్  
`generate` ఒక పరిమిత ప్రాంప్ట్‌ను నిర్మిస్తుంది (సిస్టమ్ కేవలం సందర్భాన్ని ఉపయోగించమని సూచిస్తుంది) మరియు స్థానిక మోడల్‌ను పిలుస్తుంది. తక్కువ ఉష్ణోగ్రత (0.1) సృజనాత్మకత కంటే విశ్వసనీయమైన తీయకాన్ని ప్రాధాన్యం ఇస్తుంది. త్రిమ్మింగ్ చేసిన సమాధాన పాఠ్యాన్ని తిరిగి ఇస్తుంది.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### వివరణ: ఫాల్‌బ్యాక్ క్లయింట్ ప్రారంభం
మునుపటి ప్రారంభం సెల్ దాటిపోయినా లేదా విఫలమైతే కూడా `client` ఉనికిలో ఉందని నిర్ధారిస్తుంది—తరువాతి మూల్యాంకన దశల్లో NameError ని నివారిస్తుంది.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### వివరణ: మూల్యాంకన లూప్ & మెట్రిక్స్  
మూల్యాంకన డేటాసెట్‌ను నిర్మిస్తుంది (అవసరమైన కాలమ్స్: ప్రశ్న, సమాధానం, సందర్భాలు, గ్రౌండ్_త్రూత్స్, రిఫరెన్స్) తరువాత ఎంపిక చేసిన రాగాల మెట్రిక్స్‌లపై తిరుగుతుంది.  

ఆప్టిమైజేషన్:  
- FAST_MODE వేగవంతమైన స్మోక్ టెస్టుల కోసం సమాధాన సంబంధితతకు పరిమితం చేస్తుంది.  
- ప్రతి మెట్రిక్ లూప్ ఒక మెట్రిక్ విఫలమైనప్పుడు పూర్తి పునఃగణనను నివారిస్తుంది.  

ఫలితంగా మెట్రిక్ -> స్కోరు (విఫలం అయితే NaN) డిక్ట్‌ను ఇస్తుంది.


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. అసలు పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
